In [1]:
using Distributions

In [2]:
logistic(x) = 1.0/(1.0+exp(-x))
logit(x) = -log(1.0/x - 1.0)

logit (generic function with 1 method)


# Planteamiento del problema como red neuronal


In [3]:
# cantidad de neuronas en la capa l+1
m = 2

# cantidad de neuronas en la capa l
n = 4

4

In [4]:
# algúna matriz de pesos válida para la capa l+1
W = rand(Uniform(-sqrt(n+1),sqrt(n+1)), (n+1,m))

5x2 Array{Float64,2}:
  2.17367   -2.06626 
  0.714751   0.932224
 -1.59539    0.939325
  0.310522   1.95959 
  0.811779  -0.288435

In [5]:
# algún vector válido de activaciones para la capa l
x = rand(n)

4-element Array{Float64,1}:
 0.963205
 0.195247
 0.158266
 0.545219

In [6]:
# valor de activaciones para la capa l+1 a partir de W y x
y = map(logistic, W'*[1.0;x])

2-element Array{Float64,1}:
 0.954452
 0.303208

Ahora falta encontrar una manera de generar valores aleatorios $\hat{x}$ tal que $map(logistic, W'[1.0;\hat{x}]) = y$

# Solucionando el problema con álgebra lineal

In [7]:
# A consiste de los pesos en W que estan asociados a variables desconocidas
A = W'[:,2:end]

# b consiste de la salida de la neurona l+1 menos los valores de W asociados al valor 1.0
b = y-W'[:,1]

2-element Array{Float64,1}:
 -1.21922
  2.36947

In [8]:
# Cálculo arcano para obtener la x estimada con menor norma 2
pseudoinv = pinv(A)
constant  = pseudoinv*b
factor    = eye(n)-pseudoinv*A

4x4 Array{Float64,2}:
  0.681518   0.163719  -0.42001     -0.11764   
  0.163719   0.23275   -0.135702     0.365183  
 -0.42001   -0.135702   0.265108     0.00170341
 -0.11764    0.365183   0.00170341   0.820623  

In [9]:
xLMS = constant + factor*zeros(n)

4-element Array{Float64,1}:
  0.174113
  0.806081
  0.69061 
 -0.335185

Revisamos que $xLMS$ sirve como solución

In [10]:
# debe ser muy pequeño el vector
A*xLMS - b

2-element Array{Float64,1}:
  0.0        
 -1.33227e-15

Conseguir otras soluciones

In [11]:
x2 = constant + factor*rand(n)

4-element Array{Float64,1}:
  0.692932 
  1.12818  
  0.335339 
 -0.0230787

In [12]:
A*x2 - b

2-element Array{Float64,1}:
 0.0
 0.0

Podemos realizar una busqueda sobre $w$ para encontrar una $\hat{x}$ que cumpla con las restricciones

In [13]:
function estimate(w)
    constant + factor*w
end

function estimationError(x̂)
    abs(A*x̂ - b)
end

estimationError (generic function with 1 method)

In [14]:
x̂ = estimate([.5, .5, .5, .5])

4-element Array{Float64,1}:
 0.327906
 1.11906 
 0.54616 
 0.19975 

In [15]:
estimationError(x̂)

2-element Array{Float64,1}:
 2.22045e-16
 8.88178e-16

In [62]:
middle = ones(n).*0.5

function bestOne(estimate, w1, w2, w3)
    x̂1 = estimate(w1)
    x̂2 = estimate(w2)
    x̂3 = estimate(w3)
    d1 = norm(middle-x̂1)
    d2 = norm(middle-x̂2)
    d3 = norm(middle-x̂3)
    if d1 <= d2 && d1 <= d3
        return w1
    elseif d2 <= d1 && d2 <= d3
        return w2
    elseif d3 <= d1 && d3 <= d2
        return w3
    else
        return w1
    end
end

#function getSolution(estimate, seed)
#    x̂ = estimate(seed)
#    if all(0 .<= x̂ .<= 1)
#        x̂
#    else
#        α = rand()
#        i = rand(1:length(seed))
#        w2 = copy(seed)
#        w2[i] += α
#        w3 = copy(seed)
#        w3[i] -= α
#        
#        return getSolution(estimate, bestOne(estimate, seed, w2, w3))
#    end
#end

function getSolution(estimate)
    w1 = zeros(n)
    w2 = copy(w1)
    w3 = copy(w1)
    x̂ = estimate(w1)
    
    steps = 1
    
    while true
        α = 0.00001*rand()
        i = rand(m:n)
        copy!(w1, w2)
        w2[i] =  w2[i] + α
        copy!(w1, w3)
        w3[i] = w3[i] - α
        
        x̂ = estimate(bestOne(estimate, w1, w2, w3))
        #println("$(i) >> $(x̂)")
        #sleep(.5)
        
        if all(0 .<= x̂ .<= 1) || steps == 100000
            break
        end
        steps = steps + 1
    end
    println(steps)
    return x̂
end

getSolution (generic function with 1 method)

In [63]:
x̂ = getSolution(estimate)

100000


4-element Array{Float64,1}:
  0.111883
  0.883234
  0.712389
 -0.137094

In [33]:
estimationError(x̂)

2-element Array{Float64,1}:
 3.55271e-15
 3.55271e-15

2-element Array{Float64,1}:
 1.32796 
 0.775258